In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Separar el DataFrame en dos basado en la columna 'evaluation_unit'
df_tam = df[df['evaluation_unit'].str.startswith('TAM')].copy()
df_chall = df[df['evaluation_unit'].str.startswith('CHAL')].copy()

print("DataFrame para 'TAM':")
display(df_tam.head(20))

print("\nDataFrame para 'CHALL':")
display(df_chall.head())

NameError: name 'df' is not defined

In [ ]:
df_tam.info()

In [ ]:
import pandas as pd

# Analyze percentage by diet_species for df_tam
print("Analysis of Percentage by Diet Species for TAM:")
tam_species_analysis = df_tam.groupby('diet_species')['percentage'].describe()
display(tam_species_analysis)

# Analyze percentage by diet_species for df_chall
print("\nAnalysis of Percentage by Diet Species for CHALL:")
chall_species_analysis = df_chall.groupby('diet_species')['percentage'].describe()
display(chall_species_analysis)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Cargar y Preprocesar los Datos (Asumiendo que df_tam y df_chall ya están cargados) ---
# No es necesario recargar o reasignar df aquí si ya tienes df_tam y df_chall

# Lista de DataFrames a procesar
dataframes_to_process = {'TAM': df_tam, 'CHALL': df_chall}

# --- Pasos ETL y Visualización para cada DataFrame ---

for name, current_df in dataframes_to_process.items():
    print(f"\n--- Procesando y visualizando datos para {name} ---")

    # Crear una copia para evitar SettingWithCopyWarning
    df_processed = current_df.copy()

    # --- Pasos ETL (recapitulación de lo que el informe recomienda y asumo que ya hiciste) ---
    # Normalizar nombres de columnas (ya deberían estar normalizados si provienen de df original)
    # df_processed.columns = ['evaluation_unit', 'diet_species', 'percentage'] # Esto no debería ser necesario si ya están bien

    # Convertir 'percentage' a numérico (reemplazando comas por puntos)
    # Verificamos si la columna es de tipo object antes de aplicar .str
    if df_processed['percentage'].dtype == 'object':
        df_processed['percentage'] = df_processed['percentage'].str.replace(',', '.').astype(float)
    elif df_processed['percentage'].dtype != 'float64': # Convert if not already float
         df_processed['percentage'] = df_processed['percentage'].astype(float)


    # Opcional: Tratar outliers si hay <0 o >100 (convertirlos a NaN)
    df_processed.loc[(df_processed['percentage'] < 0) | (df_processed['percentage'] > 100), 'percentage'] = np.nan
    # Opcional: Si hay NaNs después de esto y quieres eliminarlos o imputarlos:
    # df_processed.dropna(subset=['percentage'], inplace=True) # Eliminar filas con NaN en percentage
    # df_processed['percentage'].fillna(df_processed['percentage'].median(), inplace=True) # Imputar con la mediana


    print(f"¡DataFrame '{name}' listo para visualizar!")
    print(df_processed.head())
    print(f"\nEstadísticos descriptivos de 'percentage' para '{name}' después de ETL:")
    print(df_processed['percentage'].describe())

    # --- 4) Visualización (generando las gráficas) ---

    # Configuración general para las gráficas
    sns.set_style("whitegrid")
    plt.rcParams['figure.dpi'] = 100 # Mejorar la resolución en Colab

    # 1. Histograma para 'percentage'
    plt.figure(figsize=(10, 6))
    sns.histplot(df_processed['percentage'], bins=30, kde=True, color='skyblue')
    plt.title(f'Distribución de Porcentaje de Especies en la Dieta - {name}')
    plt.xlabel('Porcentaje (%)')
    plt.ylabel('Frecuencia')
    plt.show()

    # 2. Boxplot para 'percentage'
    plt.figure(figsize=(8, 6))
    sns.boxplot(y=df_processed['percentage'], color='lightcoral')
    plt.title(f'Boxplot del Porcentaje de Especies en la Dieta - {name}')
    plt.ylabel('Porcentaje (%)')
    plt.show()

    # 3. Gráfico de barras: Frecuencia de 'evaluation_unit'
    plt.figure(figsize=(12, 6))
    ax = df_processed['evaluation_unit'].value_counts().plot(kind='bar', color='lightgreen')
    plt.title(f'Frecuencia de Unidades de Evaluación - {name}')
    plt.xlabel('Unidad de Evaluación')
    plt.ylabel('Recuento')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    # Add value labels
    for p in ax.patches:
        ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 5), textcoords='offset points')
    plt.show()

    # 4. Boxplots de 'percentage' por 'evaluation_unit' (si hay suficientes unidades únicas)
    if df_processed['evaluation_unit'].nunique() > 1:
        plt.figure(figsize=(14, 7))
        sns.boxplot(x='evaluation_unit', y='percentage', data=df_processed, palette='viridis')
        plt.title(f'Distribución de Porcentaje por Unidad de Evaluación - {name}')
        plt.xlabel('Unidad de Evaluación')
        plt.ylabel('Porcentaje (%)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        print(f"Solo hay una unidad de evaluación única en {name}, no se genera Boxplot por unidad.")


    # 5. Gráfico de barras: Frecuencia de las Top 10 'diet_species'
    plt.figure(figsize=(14, 7))
    top_10_species = df_processed['diet_species'].value_counts().head(10).index
    ax = sns.countplot(y='diet_species', data=df_processed, order=top_10_species, palette='plasma')
    plt.title(f'Top 10 Especies de Dieta más Frecuentes - {name}')
    plt.xlabel('Recuento')
    plt.ylabel('Especie de Dieta')
    plt.tight_layout()
    # Add value labels
    for p in ax.patches:
        ax.annotate(f'{p.get_width()}', (p.get_width(), p.get_y() + p.get_height() / 2.),
                    ha='left', va='center', xytext=(5, 0), textcoords='offset points')
    plt.show()

    # 6. Boxplots de 'percentage' por 'diet_species' (para las Top 10)
    if not top_10_species.empty:
        plt.figure(figsize=(16, 8))
        sns.boxplot(x='diet_species', y='percentage', data=df_processed[df_processed['diet_species'].isin(top_10_species)], palette='cividis')
        plt.title(f'Distribución de Porcentaje por Top 10 Especies de Dieta - {name}')
        plt.xlabel('Especie de Dieta')
        plt.ylabel('Porcentaje (%)')
        plt.xticks(rotation=60, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        print(f"No hay suficientes especies para mostrar las Top 10 en {name}.")

In [ ]:
# Combine the processed dataframes for easier plotting
df_combined = pd.concat([df_processed.assign(group='CHALL'), df_tam.assign(group='TAM')])

# Calculate the mean percentage for each species in each group
mean_percentage = df_combined.groupby(['group', 'diet_species'])['percentage'].mean().reset_index()

# Calculate overall mean percentage for sorting
overall_mean_percentage = df_combined.groupby('diet_species')['percentage'].mean().sort_values(ascending=False).index

# Create the bar plot including all species, ordered by overall mean percentage
plt.figure(figsize=(20, 8)) # Increased figure size to accommodate more bars
ax = sns.barplot(x='diet_species', y='percentage', hue='group', data=mean_percentage, palette='viridis', order=overall_mean_percentage)
plt.title('Mean Percentage of Diet Species by Group (TAM vs CHALL)')
plt.xlabel('Diet Species')
plt.ylabel('Mean Percentage (%)')
plt.xticks(rotation=90, ha='right') # Rotated labels for readability
plt.tight_layout()

# Add value labels
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', label_type='edge')

plt.show()

# 01 Assessment of grassland condition, plant species present in microhistological slides of vicuña diet


In [ ]:
# Combine the processed dataframes for easier plotting
df_combined = pd.concat([df_processed.assign(group='CHALL')])

# Calculate the mean percentage for each species in each group
mean_percentage = df_combined.groupby(['group', 'diet_species'])['percentage'].mean().reset_index()

# Calculate overall mean percentage for sorting
overall_mean_percentage = df_combined.groupby('diet_species')['percentage'].mean().sort_values(ascending=False).index

# Create the bar plot including all species, ordered by overall mean percentage
plt.figure(figsize=(20, 8)) # Increased figure size to accommodate more bars
ax = sns.barplot(x='diet_species', y='percentage', hue='group', data=mean_percentage, palette='viridis', order=overall_mean_percentage)
plt.title('Mean Percentage of Diet Species by Group ( CHALL)')
plt.xlabel('Diet Species')
plt.ylabel('Mean Percentage (%)')
plt.xticks(rotation=90, ha='right') # Rotated labels for readability
plt.tight_layout()

# Add value labels
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', label_type='edge')

plt.show()

In [ ]:
# Combine the processed dataframes for easier plotting
df_combined = pd.concat([df_tam.assign(group='TAM')])

# Calculate the mean percentage for each species in each group
mean_percentage = df_combined.groupby(['group', 'diet_species'])['percentage'].mean().reset_index()

# Calculate overall mean percentage for sorting
overall_mean_percentage = df_combined.groupby('diet_species')['percentage'].mean().sort_values(ascending=False).index

# Create the bar plot including all species, ordered by overall mean percentage
plt.figure(figsize=(20, 8)) # Increased figure size to accommodate more bars
ax = sns.barplot(x='diet_species', y='percentage', hue='group', data=mean_percentage, palette='viridis', order=overall_mean_percentage)
plt.title('Mean Percentage of Diet Species by Group (TAM )')
plt.xlabel('Diet Species')
plt.ylabel('Mean Percentage (%)')
plt.xticks(rotation=90, ha='right') # Rotated labels for readability
plt.tight_layout()

# Add value labels
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', label_type='edge')

plt.show()

In [ ]:
df_filtered = pd.read_csv('/content/05dataa.csv')

In [ ]:
# Define the list of values to remove from the 'Record' column
values_to_remove = ['Feces', 'Lichen', 'Litter', 'Moss', 'Rock', 'Soil']

# Filter the df_data DataFrame to exclude rows with the specified 'Record' values
df_data= df_filtered[~df_filtered['Record'].isin(values_to_remove)].copy()

# Define the list of columns to drop
columns_to_drop = [f'Unnamed: {i}' for i in range(8, 26)]

# Drop the specified columns from the filtered DataFrame
df_data = df_data.drop(columns=columns_to_drop)

# Display the head and info of the new filtered DataFrame to show the result
print("DataFrame after removing specified Record categories and empty columns:")
display(df_data.head())
print("\nInfo of the updated DataFrame:")
df_data.info()

In [ ]:
df_filtered.info()


# 0505 Assessment of grassland condition, Surface transect ring-survey records (1).ipynb

## Split dataframe

### Subtask:
Split the `df_data` DataFrame into `df_data_tam` and `df_data_chall` based on the 'Evaluation_unit' column.


**Reasoning**:
Split the dataframe into two based on the 'Evaluation_unit' column and display the head of each new dataframe.



In [ ]:
# Group the DataFrame by 'Evaluation_unit' and create a dictionary of DataFrames
grouped_dataframes = {unit: df_data[df_data['Evaluation_unit'] == unit].copy()
                      for unit in df_data['Evaluation_unit'].unique()}

# Display the head of the first few generated DataFrames to show the result
print("Generated DataFrames per Evaluation Unit:")
for i, (unit, df) in enumerate(grouped_dataframes.items()):
    if i < 3: # Displaying head of first 3 for brevity
        print(f"\nDataFrame for '{unit}':")
        display(df.head())
    elif i == 3:
        print("\n... displaying head of a few more ...")
        display(df.head())
    elif i > 3 and i < 6:
         print(f"\nDataFrame for '{unit}':")
         display(df.head())
    elif i == 6:
        print("\n... and so on for all unique evaluation units.")
        break # Stop displaying after a few examples

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Loop through each DataFrame in the grouped_dataframes dictionary
for unit, df in grouped_dataframes.items():
    print(f"\n--- Analyzing Record Categories for {unit} ---")

    # Count the occurrences of each category in the 'Record' column for the current DataFrame
    record_counts = df['Record'].value_counts()

    # Check if there are any records to plot
    if record_counts.empty:
        print(f"No record categories found for {unit}.")
        continue

    # Create a bar plot (histogram of counts)
    plt.figure(figsize=(14, 7))
    ax = sns.barplot(x=record_counts.index, y=record_counts.values, palette='viridis')
    plt.title(f'Frequency of Record Categories for {unit} (Decreasing Order)')
    plt.xlabel('Record Category')
    plt.ylabel('Count')
    plt.xticks(rotation=90, ha='right') # Rotate labels for readability
    plt.tight_layout()

    # Add value labels
    for p in ax.patches:
        ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 5), textcoords='offset points')

    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Loop through each DataFrame in the grouped_dataframes dictionary
for unit, df in grouped_dataframes.items():
    print(f"\n--- Analyzing Record Categories Percentage for {unit} ---")

    # Calculate the percentage of each category in the 'Record' column for the current DataFrame
    record_percentages = df['Record'].value_counts(normalize=True) * 100

    # Check if there are any records to plot
    if record_percentages.empty:
        print(f"No record categories found for {unit}.")
        continue

    # Create a bar plot of percentages
    plt.figure(figsize=(14, 7))
    ax = sns.barplot(x=record_percentages.index, y=record_percentages.values, palette='viridis')
    plt.title(f'Percentage of Record Categories for {unit} (Decreasing Order)')
    plt.xlabel('Record Category')
    plt.ylabel('Percentage (%)')
    plt.xticks(rotation=90, ha='right') # Rotate labels for readability
    plt.tight_layout()

    # Add value labels
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.1f}%', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 5), textcoords='offset points')

    plt.show()

#04 Assessment of grassland condition, Forage availability



In [17]:
data03_df = pd.read_excel('/content/04data.xlsx')

In [18]:
data03_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Evaluation_unit  135 non-null    object
 1   Transect         135 non-null    object
 2   Date             135 non-null    object
 3   Cover            135 non-null    object
 4   Subunit          135 non-null    object
 5   Cover_perc       135 non-null    int64 
 6   Fresh_weight     135 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 7.5+ KB


In [19]:
data03_df.head(15)

,Evaluation_unit,Transect,Date,Cover,Subunit,Cover_perc,Fresh_weight
0,TAM-18,SO,2024-11-06 00:00:00,medium,T1,48,84
1,TAM-18,SO,2024-11-06 00:00:00,medium,T2,40,49
2,TAM-18,SO,2024-11-06 00:00:00,medium,T3,36,38
3,TAM-18,SO,2024-11-06 00:00:00,medium,T4,40,59
4,TAM-18,SO,2024-11-06 00:00:00,medium,T5,50,49
5,TAM-18,NO,2024-11-06 00:00:00,medium,T1,40,50
6,TAM-18,NO,2024-11-06 00:00:00,medium,T2,30,44
7,TAM-18,NO,2024-11-06 00:00:00,medium,T3,20,26
8,TAM-18,NO,2024-11-06 00:00:00,medium,T4,0,0
9,TAM-18,NO,2024-11-06 00:00:00,medium,T5,46,46


In [20]:
# Separar el DataFrame data03_df en dos basado en la columna 'Evaluation_unit'
data03_df_tam = data03_df[data03_df['Evaluation_unit'].str.startswith('TAM')].copy()
data03_df_chall = data03_df[data03_df['Evaluation_unit'].str.startswith('CHAL')].copy()

print("DataFrame para 'TAM' (data03_df_tam):")
display(data03_df_tam.head(10))

print("\nDataFrame para 'CHALL' (data03_df_chall):")


display(data03_df_chall.head())

DataFrame para 'TAM' (data03_df_tam):


,Evaluation_unit,Transect,Date,Cover,Subunit,Cover_perc,Fresh_weight
0,TAM-18,SO,2024-11-06 00:00:00,medium,T1,48,84
1,TAM-18,SO,2024-11-06 00:00:00,medium,T2,40,49
2,TAM-18,SO,2024-11-06 00:00:00,medium,T3,36,38
3,TAM-18,SO,2024-11-06 00:00:00,medium,T4,40,59
4,TAM-18,SO,2024-11-06 00:00:00,medium,T5,50,49
5,TAM-18,NO,2024-11-06 00:00:00,medium,T1,40,50
6,TAM-18,NO,2024-11-06 00:00:00,medium,T2,30,44
7,TAM-18,NO,2024-11-06 00:00:00,medium,T3,20,26
8,TAM-18,NO,2024-11-06 00:00:00,medium,T4,0,0
9,TAM-18,NO,2024-11-06 00:00:00,medium,T5,46,46



DataFrame para 'CHALL' (data03_df_chall):


,Evaluation_unit,Transect,Date,Cover,Subunit,Cover_perc,Fresh_weight
15,CHAL-01,SO,6/13/2024,low,T1,55,117
16,CHAL-01,SO,6/13/2024,low,T2,15,45
17,CHAL-01,SO,6/13/2024,low,T3,50,157
18,CHAL-01,SO,6/13/2024,low,T4,20,84
19,CHAL-01,SO,6/13/2024,low,T5,75,275


In [21]:
#sacar el promedio del data03_df_tam
print("DataFrame 'data03_df_chall' antes de la normalización:")
data03_df_chall.describe()


DataFrame 'data03_df_chall' antes de la normalización:


,Cover_perc,Fresh_weight
count,90.000000,90.000000
mean,36.755556,164.300000
std,18.058775,190.419316
min,8.000000,6.000000
25%,21.250000,65.750000
50%,36.500000,129.000000
75%,50.000000,182.000000
max,80.000000,1419.000000


In [22]:
print("DataFrame 'data03_df_tam' antes de la normalización:")
data03_df_tam.describe()

DataFrame 'data03_df_tam' antes de la normalización:


,Cover_perc,Fresh_weight
count,45.000000,45.000000
mean,45.111111,166.777778
std,18.509484,119.450425
min,0.000000,0.000000
25%,35.000000,66.000000
50%,45.000000,155.000000
75%,55.000000,231.000000
max,88.000000,520.000000


In [23]:
from sklearn.preprocessing import MinMaxScaler

# Select the numerical columns for normalization
numerical_cols = ['Cover_perc', 'Fresh_weight']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply Min-Max scaling to the numerical columns
data03_df_tam[numerical_cols] = scaler.fit_transform(data03_df_tam[numerical_cols])
data03_df_chall[numerical_cols] = scaler.transform(data03_df_chall[numerical_cols])

# Display the head of the DataFrame after normalization
print("DataFrame 'data03_df_tam' después de la normalización:")
display(data03_df_tam.head())

print("\nDataFrame 'data03_df_chall' después de la normalización:")
display(data03_df_chall.head())

DataFrame 'data03_df_tam' después de la normalización:


,Evaluation_unit,Transect,Date,Cover,Subunit,Cover_perc,Fresh_weight
0,TAM-18,SO,2024-11-06 00:00:00,medium,T1,0.545455,0.161538
1,TAM-18,SO,2024-11-06 00:00:00,medium,T2,0.454545,0.094231
2,TAM-18,SO,2024-11-06 00:00:00,medium,T3,0.409091,0.073077
3,TAM-18,SO,2024-11-06 00:00:00,medium,T4,0.454545,0.113462
4,TAM-18,SO,2024-11-06 00:00:00,medium,T5,0.568182,0.094231



DataFrame 'data03_df_chall' después de la normalización:


,Evaluation_unit,Transect,Date,Cover,Subunit,Cover_perc,Fresh_weight
15,CHAL-01,SO,6/13/2024,low,T1,0.625000,0.225000
16,CHAL-01,SO,6/13/2024,low,T2,0.170455,0.086538
17,CHAL-01,SO,6/13/2024,low,T3,0.568182,0.301923
18,CHAL-01,SO,6/13/2024,low,T4,0.227273,0.161538
19,CHAL-01,SO,6/13/2024,low,T5,0.852273,0.528846


sacar el promedio del data03_df_tam (TAMBOKARKAS)





In [24]:
#sacar el promedio del data03_df_tam
data03_df_tam.describe()

,Cover_perc,Fresh_weight
count,45.000000,45.000000
mean,0.512626,0.320726
std,0.210335,0.229712
min,0.000000,0.000000
25%,0.397727,0.126923
50%,0.511364,0.298077
75%,0.625000,0.444231
max,1.000000,1.000000




---



sacar el promedio del data03_df_chall (challalpata)





In [25]:
#sacar el promedio del data02_df_tam
data03_df_chall.describe()

,Cover_perc,Fresh_weight
count,90.000000,90.000000
mean,0.417677,0.315962
std,0.205213,0.366191
min,0.090909,0.011538
25%,0.241477,0.126442
50%,0.414773,0.248077
75%,0.568182,0.350000
max,0.909091,2.728846


CONCLUSIONES


Mayor Disponibilidad de Forraje en Chalhuani (CHAL) 🌿
Chalhuani es, en promedio, un sitio más productivo y con mayor cobertura que Tambojarkas.

Biomasa (fresh_weight_norm): El promedio en CHAL es positivo (0.137), lo que indica que su biomasa está por encima de la media general del estudio. En contraste, el promedio en TAM es negativo (-0.198), situándose por debajo de la media general.

Cobertura (cover_perc_norm): Se repite el mismo patrón. El promedio de cobertura en CHAL es positivo (0.052), mientras que en TAM es negativo (-0.075).

Inferencia: Si el objetivo es encontrar forraje, Chalhuani es consistentemente el mejor lugar. Ofrece más pasto tanto en extensión (cobertura) como en cantidad (biomasa).

Desviación Estándar (std) de la Biomasa: Este es el dato más revelador. La variabilidad en TAM es significativamente mayor (1.14) que en CHAL (0.83).

Valores Extremos (min y max):

Ambos sitios tienen un mínimo idéntico (-0.92), lo que significa que ambos tienen zonas sin biomasa.

Sin embargo, el valor máximo de biomasa en TAM (2.87) es mucho más extremo que el máximo en CHAL (1.95).

**Inferencia:** Tambojarkas se puede describir como un pastizal "parcheado". La mayoría de sus áreas tienen poca biomasa (el 75% de sus datos están apenas por encima de la media general, en 0.009), pero existen unos pocos "puntos calientes" o "hotspots" con una biomasa excepcionalmente alta. Chalhuani, por otro lado, es más uniforme y predecible en su productividad.

Chalhuani (CHAL): Es el sitio más confiable y productivo. Ofrece una buena cantidad de forraje de manera consistente a lo largo de su extensión. Es ideal para un pastoreo sostenido.

Tambojarkas (TAM): Es un sitio de alto riesgo y alta recompensa. En general es pobre en forraje, pero contiene focos aislados de muy alta productividad. El comportamiento de los animales (como las vicuñas) podría ser de búsqueda intensiva para localizar estos "parches" valiosos.

#02 Assessment of grassland condition, biomass and digestibility of plants





In [26]:
data02_df = pd.read_excel('/content/02data.xlsx')

In [27]:
data02_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Evaluation_unit  27 non-null     object 
 1   Transect         27 non-null     object 
 2   Fresh_weight     27 non-null     int64  
 3   Dry_weight       27 non-null     int64  
 4   Dry_matter       27 non-null     float64
 5   Digestible_leaf  27 non-null     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 1.4+ KB


In [11]:
data02_df.head(15)

,Evaluation_unit,Transect,Fresh_weight,Dry_weight,Dry_matter,Digestible_leaf
0,TAM-18,SO,250,220,12.00,88.00
1,TAM-18,SE,250,199,20.40,79.60
2,CHAL-01,SO,250,188,24.80,75.20
3,CHAL-01,NO,194,143,26.29,73.71
4,TAM-15,SE,250,188,24.80,75.20
5,TAM-18,NO,166,123,25.90,74.10
6,TAM-15,NO,250,192,23.20,76.80
7,TAM-15,SO,250,204,18.40,81.60
8,CHAL-01,SE,250,200,20.00,80.00
9,CHAL-04,SO,250,180,28.00,72.00


In [12]:
# Separar el DataFrame data02_df en dos basado en la columna 'Evaluation_unit'
data02_df_tam = data02_df[data02_df['Evaluation_unit'].str.startswith('TAM')].copy()
data02_df_chall = data02_df[data02_df['Evaluation_unit'].str.startswith('CHAL')].copy()

print("DataFrame para 'TAM' (data02_df_tam):")
display(data02_df_tam.head(10))

print("\nDataFrame para 'CHALL' (data02_df_chall):")


display(data02_df_chall.head())

DataFrame para 'TAM' (data02_df_tam):


,Evaluation_unit,Transect,Fresh_weight,Dry_weight,Dry_matter,Digestible_leaf
0,TAM-18,SO,250,220,12.0,88.0
1,TAM-18,SE,250,199,20.4,79.6
4,TAM-15,SE,250,188,24.8,75.2
5,TAM-18,NO,166,123,25.9,74.1
6,TAM-15,NO,250,192,23.2,76.8
7,TAM-15,SO,250,204,18.4,81.6
10,TAM-17,SE,250,183,26.8,73.2
13,TAM-17,SO,250,186,25.6,74.4
14,TAM-17,NO,250,186,25.6,74.4



DataFrame para 'CHALL' (data02_df_chall):


,Evaluation_unit,Transect,Fresh_weight,Dry_weight,Dry_matter,Digestible_leaf
2,CHAL-01,SO,250,188,24.80,75.20
3,CHAL-01,NO,194,143,26.29,73.71
8,CHAL-01,SE,250,200,20.00,80.00
9,CHAL-04,SO,250,180,28.00,72.00
11,CHAL-04,NO,250,174,30.40,69.60


In [13]:
#sacar el promedio del data02_df_tam
data02_df_tam.describe()

,Fresh_weight,Dry_weight,Dry_matter,Digestible_leaf
count,9.000000,9.000000,9.000000,9.000000
mean,240.666667,186.777778,22.522222,77.477778
std,28.000000,26.602527,4.828503,4.828503
min,166.000000,123.000000,12.000000,73.200000
25%,250.000000,186.000000,20.400000,74.400000
50%,250.000000,188.000000,24.800000,75.200000
75%,250.000000,199.000000,25.600000,79.600000
max,250.000000,220.000000,26.800000,88.000000


In [14]:
data02_df_chall.describe()

,Fresh_weight,Dry_weight,Dry_matter,Digestible_leaf
count,18.000000,18.000000,18.000000,18.000000
mean,246.888889,173.166667,29.816111,70.183889
std,13.199327,22.276999,8.432554,8.432554
min,194.000000,100.000000,20.000000,40.000000
25%,250.000000,169.500000,25.400000,68.700000
50%,250.000000,175.000000,29.000000,71.000000
75%,250.000000,186.500000,31.300000,74.600000
max,250.000000,200.000000,60.000000,80.000000


In [28]:
from sklearn.preprocessing import MinMaxScaler

# Select the numerical columns for normalization
numerical_cols = ['Fresh_weight', 'Dry_weight', 'Dry_matter', 'Digestible_leaf']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply Min-Max scaling to the numerical columns
data02_df_tam[numerical_cols] = scaler.fit_transform(data02_df_tam[numerical_cols])
data02_df_chall[numerical_cols] = scaler.transform(data02_df_chall[numerical_cols])

# Display the head of the DataFrame after normalization
print("DataFrame 'data02_df_tam' después de la normalización:")
display(data02_df_tam.head())

print("\nDataFrame 'data02_df_chall' después de la normalización:")
display(data02_df_chall.head())

DataFrame 'data02_df_tam' después de la normalización:


,Evaluation_unit,Transect,Fresh_weight,Dry_weight,Dry_matter,Digestible_leaf
0,TAM-18,SO,1.0,1.000000,0.000000,1.000000
1,TAM-18,SE,1.0,0.783505,0.567568,0.432432
4,TAM-15,SE,1.0,0.670103,0.864865,0.135135
5,TAM-18,NO,0.0,0.000000,0.939189,0.060811
6,TAM-15,NO,1.0,0.711340,0.756757,0.243243



DataFrame 'data02_df_chall' después de la normalización:


,Evaluation_unit,Transect,Fresh_weight,Dry_weight,Dry_matter,Digestible_leaf
2,CHAL-01,SO,1.000000,0.670103,0.864865,0.135135
3,CHAL-01,NO,0.333333,0.206186,0.965541,0.034459
8,CHAL-01,SE,1.000000,0.793814,0.540541,0.459459
9,CHAL-04,SO,1.000000,0.587629,1.081081,-0.081081
11,CHAL-04,NO,1.000000,0.525773,1.243243,-0.243243


sacar el promedio del data02_df_tam (TAMBOKARKAS)





In [29]:
#sacar el promedio del data02_df_tam
data02_df_tam.describe()

,Fresh_weight,Dry_weight,Dry_matter,Digestible_leaf
count,9.000000,9.000000,9.000000,9.000000
mean,0.888889,0.657503,0.710961,0.289039
std,0.333333,0.274253,0.326250,0.326250
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.649485,0.567568,0.081081
50%,1.000000,0.670103,0.864865,0.135135
75%,1.000000,0.783505,0.918919,0.432432
max,1.000000,1.000000,1.000000,1.000000




---



sacar el promedio del data02_df_tam (challalpata)





In [30]:
#sacar el promedio del data02_df_tam
data02_df_chall.describe()

,Fresh_weight,Dry_weight,Dry_matter,Digestible_leaf
count,18.000000,18.000000,18.000000,18.000000
mean,0.962963,0.517182,1.203791,-0.203791
std,0.157135,0.229660,0.569767,0.569767
min,0.333333,-0.237113,0.540541,-2.243243
25%,1.000000,0.479381,0.905405,-0.304054
50%,1.000000,0.536082,1.148649,-0.148649
75%,1.000000,0.654639,1.304054,0.094595
max,1.000000,0.793814,3.243243,0.459459


CONCLUSIONES

**El Forraje de Tambojarkas (TAM) es de Mayor Calidad Nutricional**

Tambojarkas, a pesar de tener menos biomasa (como vimos en el análisis anterior), ofrece un pasto más nutritivo y digestible.

Hoja Digestible (Digestible_leaf): El promedio en TAM es 77.2%, notablemente superior al promedio de CHAL (72.6%). Esto significa que por cada gramo de pasto seco, las vicuñas pueden aprovechar más nutrientes en Tambojarkas.

Materia Seca (Dry_matter): El promedio en TAM es más bajo (22.8%) que en CHAL (27.4%). Un menor porcentaje de materia seca suele estar asociado a pastos más tiernos, menos fibrosos y, por tanto, más fáciles de digerir.

Inferencia: Hay una clara compensación (trade-off) entre cantidad y calidad. Chalhuani ofrece cantidad, pero Tambojarkas ofrece calidad.

**La Calidad en Chalhuani (CHAL) es Más Variable**

La calidad del forraje en Chalhuani no solo es menor en promedio, sino también más inconsistente.

Desviación Estándar (std) de la Hoja Digestible: La variabilidad en CHAL es mayor (3.23) que en TAM (2.89).

Rango (min y max): El rango de digestibilidad en Chalhuani es más amplio, con un mínimo de 69.6%, que es un valor de calidad bastante bajo en comparación con el mínimo de Tambojarkas (73.2%).

**Inferencia**: En Chalhuani, los animales pueden encontrar tanto parches de forraje de calidad decente como parches de calidad pobre. En Tambojarkas, la calidad es más consistente y predeciblemente alta.

**Resumen Ejecutivo y Recomendación 💡**

Chalhuani (CHAL): Se caracteriza por tener alta cantidad de biomasa pero de menor y más variable calidad nutricional. Es un sitio de "volumen".

Tambojarkas (TAM): Se caracteriza por tener poca biomasa, pero de alta y consistente calidad nutricional. Es un sitio de "calidad".

Esta dualidad es clave para entender el ecosistema. Las vicuñas podrían emplear estrategias de pastoreo diferentes en cada sitio: en Chalhuani para saciarse ("llenar el estómago") y en Tambojarkas para obtener nutrientes clave, quizás de forma más selectiva.

# 06 Assessment of grassland condition, Indicator plants vigor


In [31]:
data06_df = pd.read_excel('/content/06data.xlsx')

In [32]:
data06_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Numeration         297 non-null    int64         
 1   Evaluation_unit    297 non-null    object        
 2   Transect           297 non-null    object        
 3   Cover              297 non-null    object        
 4   Date               297 non-null    datetime64[ns]
 5   Family             296 non-null    object        
 6   Genus              296 non-null    object        
 7   Indicator_species  297 non-null    object        
 8   Plant_vigor        297 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 21.0+ KB


In [33]:
data06_df.head(15)

,Numeration,Evaluation_unit,Transect,Cover,Date,Family,Genus,Indicator_species,Plant_vigor
0,1,TAM-18,NO,medium,2024-06-11,Poaceae,Cinnagrostis,Cinnagrostis_vicunarum,9
1,2,TAM-18,NO,medium,2024-06-11,Poaceae,Cinnagrostis,Cinnagrostis_vicunarum,14
2,3,TAM-18,NO,medium,2024-06-11,Poaceae,Festuca,Festuca_peruviana,10
3,4,TAM-18,NO,medium,2024-06-11,Poaceae,Festuca,Festuca_peruviana,16
4,5,TAM-18,NO,medium,2024-06-11,Poaceae,Cinnagrostis,Cinnagrostis_vicunarum,8
5,6,TAM-18,NO,medium,2024-06-11,Poaceae,Cinnagrostis,Cinnagrostis_vicunarum,9
6,7,TAM-18,NO,medium,2024-06-11,Poaceae,Festuca,Festuca_peruviana,18
7,8,TAM-18,NO,medium,2024-06-11,Poaceae,Cinnagrostis,Cinnagrostis_vicunarum,5
8,9,TAM-18,NO,medium,2024-06-11,Poaceae,Cinnagrostis,Cinnagrostis_vicunarum,7
9,10,TAM-18,NO,medium,2024-06-11,Poaceae,Festuca,Festuca_peruviana,19


In [34]:
# Group by 'Indicator_species'
grouped_by_species = data06_df.groupby('Indicator_species')

# Display the groups (showing the first few rows of each group for brevity)
print("Data grouped by Indicator Species:")
for species, group_df in grouped_by_species:
    print(f"\nSpecies: {species}")
    display(group_df.head())

Data grouped by Indicator Species:

Species: Alchemilla_pinnata


,Numeration,Evaluation_unit,Transect,Cover,Date,Family,Genus,Indicator_species,Plant_vigor
53,54,CHAL-01,SO,low,2024-06-13,Rosaceae,Alchemilla,Alchemilla_pinnata,1
54,55,CHAL-01,SO,low,2024-06-13,Rosaceae,Alchemilla,Alchemilla_pinnata,2



Species: Cinnagrostis_curvula


,Numeration,Evaluation_unit,Transect,Cover,Date,Family,Genus,Indicator_species,Plant_vigor
90,91,CHAL-04,NO,medium,2024-06-17,Poaceae,Cinnagrostis,Cinnagrostis_curvula,12



Species: Cinnagrostis_minima


...

,Numeration,Evaluation_unit,Transect,Cover,Date,Family,Genus,Indicator_species,Plant_vigor
2,3,TAM-18,NO,medium,2024-06-11,Poaceae,Festuca,Festuca_peruviana,10
3,4,TAM-18,NO,medium,2024-06-11,Poaceae,Festuca,Festuca_peruviana,16
6,7,TAM-18,NO,medium,2024-06-11,Poaceae,Festuca,Festuca_peruviana,18
9,10,TAM-18,NO,medium,2024-06-11,Poaceae,Festuca,Festuca_peruviana,19
10,11,TAM-18,SO,medium,2024-06-11,Poaceae,Festuca,Festuca_peruviana,18



Species: Trichophorum_rigidum


,Numeration,Evaluation_unit,Transect,Cover,Date,Family,Genus,Indicator_species,Plant_vigor
60,61,CHAL-03,NO,low,2024-06-16,Cyperaceae,Trichophorum,Trichophorum_rigidum,10
67,68,CHAL-03,NO,low,2024-06-16,Cyperaceae,Trichophorum,Trichophorum_rigidum,9
71,72,CHAL-03,SE,low,2024-06-16,Cyperaceae,Trichophorum,Trichophorum_rigidum,7
72,73,CHAL-03,SE,low,2024-06-16,Cyperaceae,Trichophorum,Trichophorum_rigidum,6
73,74,CHAL-03,SE,low,2024-06-16,Cyperaceae,Trichophorum,Trichophorum_rigidum,8



Species: cinnagrostis_vicunarum


,Numeration,Evaluation_unit,Transect,Cover,Date,Family,Genus,Indicator_species,Plant_vigor
121,122,CHAL-08,NO,low,2024-06-19,NaN,NaN,cinnagrostis_vicunarum,3


In [35]:
# Display descriptive statistics for each species group
print("Descriptive statistics for each Indicator Species group:")
for species, group_df in grouped_by_species:
    print(f"\nSpecies: {species}")
    display(group_df.describe())

Descriptive statistics for each Indicator Species group:

Species: Alchemilla_pinnata


,Numeration,Date,Plant_vigor
count,2.000000,2,2.000000
mean,54.500000,2024-06-13 00:00:00,1.500000
min,54.000000,2024-06-13 00:00:00,1.000000
25%,54.250000,2024-06-13 00:00:00,1.250000
50%,54.500000,2024-06-13 00:00:00,1.500000
75%,54.750000,2024-06-13 00:00:00,1.750000
max,55.000000,2024-06-13 00:00:00,2.000000
std,0.707107,NaN,0.707107



Species: Cinnagrostis_curvula


,Numeration,Date,Plant_vigor
count,1.0,1,1.0
mean,91.0,2024-06-17 00:00:00,12.0
min,91.0,2024-06-17 00:00:00,12.0
25%,91.0,2024-06-17 00:00:00,12.0
50%,91.0,2024-06-17 00:00:00,12.0
75%,91.0,2024-06-17 00:00:00,12.0
max,91.0,2024-06-17 00:00:00,12.0
std,NaN,NaN,NaN



Species: Cinnagrostis_minima


...

,Numeration,Date,Plant_vigor
count,141.000000,141,141.000000
mean,144.446809,2024-06-14 15:19:08.936170240,15.829787
min,3.000000,2024-06-11 00:00:00,5.000000
25%,59.000000,2024-06-12 00:00:00,10.000000
50%,136.000000,2024-06-14 00:00:00,13.000000
75%,228.000000,2024-06-17 00:00:00,18.000000
max,297.000000,2024-06-20 00:00:00,76.000000
std,93.998285,NaN,9.799386



Species: Trichophorum_rigidum


,Numeration,Date,Plant_vigor
count,41.000000,41,41.000000
mean,127.731707,2024-06-17 18:08:46.829268224,9.268293
min,61.000000,2024-06-16 00:00:00,6.000000
25%,80.000000,2024-06-16 00:00:00,8.000000
50%,142.000000,2024-06-18 00:00:00,9.000000
75%,156.000000,2024-06-19 00:00:00,11.000000
max,180.000000,2024-06-19 00:00:00,15.000000
std,38.707250,NaN,2.121608



Species: cinnagrostis_vicunarum


,Numeration,Date,Plant_vigor
count,1.0,1,1.0
mean,122.0,2024-06-19 00:00:00,3.0
min,122.0,2024-06-19 00:00:00,3.0
25%,122.0,2024-06-19 00:00:00,3.0
50%,122.0,2024-06-19 00:00:00,3.0
75%,122.0,2024-06-19 00:00:00,3.0
max,122.0,2024-06-19 00:00:00,3.0
std,NaN,NaN,NaN


CONCLUSIONES


Festuca_peruviana es la Especie de Mayor Tamaño y Variabilidad 🌾

Presenta la media más alta (20.6 cm) y el valor máximo más extremo (76 cm).

Su desviación estándar (14.6 cm) es, con diferencia, la más grande, lo que indica una enorme variabilidad en su tamaño. La mayoría de los individuos son pequeños (mediana de 16 cm), pero unos pocos ejemplares son excepcionalmente altos. Esto sugiere que su crecimiento es muy sensible a condiciones micro-locales.

Cinnagrostis_vicunarum y Muhlenbergia_fastigiata son Especies más Pequeñas y Homogéneas 🌱

Cinnagrostis tiene una media de 10.5 cm y Muhlenbergia de 12.6 cm.

Ambas muestran una variabilidad mucho menor (desviación estándar de 4.6 y 5.7 cm, respectivamente) en comparación con Festuca. Sus alturas son más consistentes y predecibles.

Cinnagrostis es notablemente la especie con el vigor más bajo en general.

Poa_gymnantha es una Especie Intermedia pero Poco Frecuente

Con una media de 15 cm y una desviación estándar de 5.1 cm, se sitúa en un punto intermedio en términos de altura y variabilidad.

Sin embargo, su conteo es muy bajo (solo 6 registros), lo que hace que sus estadísticos sean menos fiables que los de las otras especies. Las conclusiones sobre esta planta deben tomarse con cautela.

Recomendación General

El análisis debe centrarse en las diferencias entre Festuca_peruviana y Cinnagrostis_vicunarum, ya que son las especies más representativas en el muestreo y exhiben comportamientos de crecimiento claramente distintos. La alta variabilidad y los outliers en Festuca merecen una investigación más profunda: ¿están correlacionados con algún factor específico como el cover, el transect o la evaluation_unit? Responder a esa pregunta podría revelar qué condiciones favorecen un crecimiento excepcional en esta especie clave.